In [1]:
import tensorflow_text

In [2]:
import tensorflow as tf

In [3]:
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocessor_url)

In [ ]:
text_test = ['This came in on time and I am very happy with it',
            
            'Works as well as the factory tool']

In [ ]:
text_preprocessed = bert_preprocess_model(text_test)

In [ ]:
text_preprocessed.keys()

In [ ]:
text_preprocessed['input_mask']

In [ ]:
text_preprocessed['input_word_ids']

In [ ]:
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
bert_result = bert_model(text_preprocessed)

In [ ]:
bert_result.keys()

In [ ]:
bert_result['pooled_output']

In [ ]:
bert_result['encoder_outputs']

## Text Classifier Spam or not

In [ ]:
import pandas as pd 

df = pd.read_csv('spam.csv')
df.head()

In [ ]:
df.Message[10]

In [ ]:
df.groupby('Category').describe()

In [ ]:
df['spam'] = df['Category'].apply(lambda x :1 if x=='spam' else 0)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test =  train_test_split(df['Message'], df['spam'], test_size=0.2, random_state=42, stratify=df['spam'])

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess_model(sentences)
    return bert_model(preprocessed_text)['pooled_output']

### Let's try out our function

In [ ]:
get_sentence_embedding([
    "I am gonna be home soon and i don't want to talk about this stuff anymore tonight",
    "You have won a lottery. please click on link to claim"
])

In [ ]:
e = get_sentence_embedding([
    "banana",
    "apple",
    "mango",
    "police",
    "lawyer",
    "dog"
])

In [ ]:
e

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

comparing vector of banana and apple

In [ ]:
cosine_similarity([e[0]], [e[1]])

In [ ]:
cosine_similarity([e[0]], [e[2]])

## Building Model

In [1]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])

final_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="final")(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[final_layer])
model.summary()

NameError: name 'tf' is not defined

In [57]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [58]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 1297s 9s/step - loss: 0.3135 - accuracy: 0.8672
Epoch 2/5
140/140 [==============================] - 1224s 9s/step - loss: 0.2217 - accuracy: 0.9096
Epoch 3/5
140/140 [==============================] - 1231s 9s/step - loss: 0.1866 - accuracy: 0.9289
Epoch 4/5
140/140 [==============================] - 1225s 9s/step - loss: 0.1619 - accuracy: 0.9417
Epoch 5/5
140/140 [==============================] - 1432s 10s/step - loss: 0.1472 - accuracy: 0.9511


In [59]:
model.evaluate(X_test,y_test)

35/35 [==============================] - 482s 14s/step - loss: 0.1428 - accuracy: 0.9632


[0.1427883505821228, 0.963228702545166]

In [6]:
sample = [
    "Reply to win $5000 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end",
    "You are awarded a SiPix Digital Camera! call 0906122061 from landline.Delivery within 28days",
    "it to 80488. Your 500 free text message are valid until 31 December 2005",
    "Hey Sam, Are you coming for a cricket game tomorrow",
    "Why don't you wait until at least wednesday to see if you get your"
]



In [69]:
def Predict(text):
    output = model.predict(text)
    result = []
    for i in output:
        if i >0.5:
            i = "spam"
            result.append(i)
        else:
            i = "not a spam"
            result.append(i)
    return result

In [70]:
Predict(sample)

1/1 [==============================] - 2s 2s/step


['spam', 'spam', 'not a spam', 'not a spam', 'not a spam']

In [78]:
model.save("./Spam_or_not_a_spam", include_optimizer=False)

INFO:tensorflow:Assets written to: ./Spam_or_not_a_spam\assets


INFO:tensorflow:Assets written to: ./Spam_or_not_a_spam\assets


In [4]:
import tensorflow as tf
import tensorflow_text as text

reloaded_model = tf.saved_model.load("./Spam_or_not_a_spam")

In [8]:
reloaded_model(sample)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.6556829 ],
       [0.6451667 ],
       [0.47689074],
       [0.04374119],
       [0.01255653]], dtype=float32)>

In [7]:
def Prediction(text):
    output = reloaded_model(text)
    result = []
    for i in output:
        if i >0.5:
            i = "spam"
            result.append(i)
        else:
            i = "not a spam"
            result.append(i)
    return result

In [9]:
Prediction(sample)

['spam', 'spam', 'not a spam', 'not a spam', 'not a spam']